### Your name:

<pre> Joan Soo Li Lim </pre>

### Collaborators:

<pre> None </pre>


Markdown reference can be found here:
    http://nestacms.com/docs/creating-content/markdown-cheat-sheet

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator

Q1. Why would it be a problem if our training set and test set are the same.

The end goal of ML is to build a model that can be generalized to new data. It would be beneficial to divide the training data into a training set and test/validation set, where the latter set represents real world data (i.e. unseen and unfitted data). Otherwise, if the training data  is reused during test/validation, then one will not be able to predict the accuracy of the model on data it has not seen before. In other words, if we train on test/validation data, then the model will appear to fit with good accuracy (as we are testing on the same data) but might not be able to generalize adequately when the ML model encounters new data. 


Q2. Explain step by step the process to select k in the k-nearest neighbor algorithm (pseudocode)

To find k using the simpliest algorithm (k with lowest error rate), the pseudocode is as follows: 

Find K-simple
1. Create an array err_rates of size n and initialize to 0 (n = number of observations in training set) **
2. Set the random seed
3. Split the number of observations into 2 sets: training (i.e. 80%) and validation (i.e. 20%)  
4. For each k in range of 1 .. square root(n) 
    a. Fit the model on the training data for k neighbours
    b. Predict the value of the validation set from the model derived
    c. Determine the error rate of validation set vs. predicted value (using something like MSE)
    d. Store the error rate in err_rates at index i == k
5. Find the minimum value of err_rates and return its index.     


Note, if we select k based on M fold validation, the pseudocode will now be encapsulated by M iterations such as:

Find K-Mfold
1. Create a matrix for cross validation errrors cross_err_rates of size n by M 
    a. Initialize to 0 
    b. Add ID column = k for range 1 .. square root(n)
    c. Add error_mean column to store mean of error for this k
2. Set the random seed
3. For each m in range of 1 .. M
        For each k in range of 1 .. square root(n)  
        a. Split the number of observations into 2 sets: training (i.e. m−1/m %) and validation (i.e. 1/m %)
        b. Fit the model on the training data for k neighbours
        c. Predict the value of the validation set from the model derived
        d. Determine the error rate of validation set vs. predicted value (using something like MSE)
        e. Store the error rate in cross_err_rates at location (k, m)
        f. Find the average value of cross_err_rates for this m and store in error_mean
4. Find the minimum value of error_mean in cross_err_rates and return the value of ID column for that row        
        

** The bound for k is set as per lecture slides (Lecture 1 slide 60).

Reference:
https://idc9.github.io/stor390/notes/cross_validation/cross_validation.html 


Q3. For the k-nearest regression. What happens when k = n. Where n is equal to the training size.

In general for a large dataset, a k that is too small may result in overfitting (i.e extreme case k==1 considers each point individually) . Conversely, a large k may result in smoothling out noise (outliers) and eliminating important details. If k==n, we have the extreme case where the whole distribution is evened out and as per the next Q below for KNN regression, we will then take the average of all the points. This averaging will lead to a straight line (high bias, no variance). In the case of classification, we will then simply take the most dominant class (i.e. If there are 2 classes, where class X appears 10 times and class Y appears 9 times, the model will return class X).

Reference:
http://www.cs.bham.ac.uk/~jxb/INC/l9.pdf (slide 13)


Q4. Define a function that takes a 1-d numpy array, a parameter k, and a number p.
The function returns an estimate equal to the mean of the closest k points to the number p

In [2]:
def k_neighbor(input_data, k, p):
    """Returns the k-neighbor estimate for p using data input_data.

    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    """
    
    # Initalize
    mean = 0
    distances = []
    neighbors = []
    
    # For each point in input_data, compute Manhattan distance to p
    for x in range(len(input_data)):
        dist = np.abs(p - input_data[x])
        distances.append((input_data[x], dist))
    
    # Sort the distances 
    distances.sort(key=operator.itemgetter(1))
    
    # Get the first k neighbours
    for x in range(k):
        neighbors.append(distances[x][0])
    
    # Compute the mean of the first k neighbours' value(s) in input_data
    mean = np.mean(neighbors)
    
    # Return the mean
    return (mean)
    

In [3]:
#Evaluate
data = np.array([1,3,4,5,7,8,11,12,13,15,19,24,25,29,40])
print(k_neighbor(input_data=data, k=3, p=5))
print(k_neighbor(input_data=data, k=2, p=15))
print(k_neighbor(input_data=data, k=3, p=25))
print(k_neighbor(input_data=data, k=1, p=55))
print(k_neighbor(input_data=data, k=3, p=55))
print(k_neighbor(input_data=data, k=10, p=55))

4.0
14.0
26.0
40.0
31.333333333333332
19.6


Some observations include: 

For k=3, p=5 -> we have the case where both 3 and 7 have a distance of 2 away from 5. Therefore, depending on the algorithmic choice, we could have picked either 3 or 7 when attempting to predict the value for p. As I chose to sort by original array range, I picked 3. The predicted value is then 4.0. If some other partitioning was chosen to sort and order the data, the value could be 5.3 (4+5+7 / 3).

For p=55 -> depending on the k, we have a huge variance of predicted values. The is b/c the original dataset has data that is primarily on the lower end but sparse for higher p. Therefore, for high p, the model cannot generalize as well as for low p.


Q5. Similar to Q4 but for the n dimentional case. 

In [4]:
def l1_norm(a,b):
    """Returns the l1 norm (a,b)"""
    return np.sum(abs(a-b))

    
def l2_norm(a,b):
    """Returns the l2 norm (a,b)""" 
    return np.sum((b-a)**2)**0.5

    
def k_neighbor_nd(input_data, k, p, metric='l1', mode='mean'):
    """Returns the k-neighbor estimate for p using data input_data.

    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    metric -- l1 or l2. l1 norm or l2 norm https://en.wikipedia.org/wiki/Norm_(mathematics)
    mode -- estimator possible values = 'mean', 'median', 'max'
    
    Implement the l1 and l2 norms
    for mean, median and max, use np.mean, np.median and np.max
    """
    
    # Initialize
    arr = np.zeros (0)
    idx = []
    neighbors = []
    predicted = []
    debug = False
    
    # For each point in input_data, compute distance to p using either l1 or l2
    if metric == 'l1':
        for i in range (data_4d.shape[0]):
           arr = np.append(arr, l1_norm (p, data_4d[i]))
    else :
        for i in range (data_4d.shape[0]):
           arr = np.append(arr, l2_norm (p, data_4d[i])) 
    
    # Sort the distances and get the first k neighbours's value(s)
    idx = np.argpartition (arr, k) # indices of input_data of first k 
    neighbors = (data_4d[idx[:k]])
    
    if debug:    
        print (arr)
        print (idx)
        print (neighbors)
    
    # Compute the predictor of the first k neighbours' value(s) in input_data using appropriate mode
    if mode == 'mean':
        predicted = np.mean(neighbors, axis = 0)   
    elif mode == 'max':
        predicted = np.max(neighbors, axis = 0)
    else :
        predicted = np.median(neighbors, axis = 0)
    
    # Return the predictor
    return predicted
   

In [5]:
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])

In [6]:
#Evaluate
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='median'))
print ("--------------------------") # Separate metric
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l2', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='median'))

[2.         2.33333333 4.         1.        ]
[4. 3. 0. 2.]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]
--------------------------
[1.33333333 2.         4.         1.        ]
[3.5 3.  0.5 1.5]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]


Some observations include:

Regardless of metric, the last 3 examples for either l1 or l2 norm return identical k neighbors. For the max, we have a predicted value that still has quite a distance from p. I suspect there could be a larger variance for large k for this mode. In terms of p = [2, 3, 3, 4] and k = 1, we simply return the closest neighbor and hence the mode is not really relevant. However, for k = 3, the predicted value with mode = median does not really provide a better error rate as the additional neighbors have quite a distance from p. 

For the first 2 examples, the metric l2 computes a more complex set of distances and returns one neighbor that is different from the metric l1. In general for these examples, l1 seems to favor neighbors with distances that minimize differences amongst each dimension, whereas l2 seems to favor neighbors with distances that minimize differences as a whole amongst all the dimensions due to errors > 1 being squared. Interestingly, if we look at the error rate between p and the predicted value, then l1 seems to be slightly better. Since I am a novice, I am not sure if higher dimensions affect KNN negatively due to this sort of distortion in the distance calculation. Perhaps dimension reduction could help. Or perhaps the l2 model could have been tweaked by picking a higher k (in order to have more neighbors to average / smooth out the predicted value). 


Q6. Read the documentation on KNeighborsRegressor

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
    
Explore the source code:
    https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/regression.py
        
How different it is from your implementation? How well can you follow the code? Did you learn something new?

The code has various portions that are difficult to follow, but overall the main idea is to permit additional tweaking on the KNN algorithm beyond that of metric and mode. In particular, there are possibilities such as adding weights (either inverse to the distance or user-defined) to data points such that those of a further distance could have less bearing on the model's fit. This notion would be beneficial for areas of a distribution that have sparse data points around a specific p (as noted in our example above). The data structures utilized for speed and memory optimization are far more advanced than my arrays and are especially needed for higher dimensions and larger input. Note, I suspect that building n-leaf trees could cause me to have a massive headache. Additionally, the default distance computed is Minkowski which generalized to p = 1 (l1) and p = 2 (l2). I am not sure what the results would be for higher powers of p and its relvance as it approaches infinity, but perhaps this will be covered later.  There are other minor details such as the ability to return the distances and indices / values of the neighbors (which I have implemented indirectly via a debug boolean). 